In [ ]:

import json
import seaborn as sns
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
from pathlib import Path
from binoculars.detector import BINOCULARS_ACCURACY_THRESHOLD, BINOCULARS_FPR_THRESHOLD
from scipy import stats
from scipy.stats import pearsonr
from run_analysis import load_chatgpt_trends, cr_has_en, has_en_mapping, p2mark
import matplotlib.colors as mcolors
import seaborn as sns
import joblib
from statannotations.Annotator import Annotator
from pySankey.sankey import sankey
import matplotlib.cm as cm
import statsmodels.api as sm
from statsmodels.formula.api import ols, glm
from pingouin import ancova
import tiktoken
import openai
from tqdm import tqdm
from semanticscholar import SemanticScholar
sns.set_theme(style="ticks")
gpt_release_date = pd.to_datetime('2022-11-30')

In [ ]:
df_bypaper_scored = pd.read_pickle('data/df_bypaper_scored.pkl')
df_bypaper_scored['n_tokens_k'] = df_bypaper_scored['n_tokens']//1000
df_bypaper_scored.describe()

paperinfo_df = pd.read_hdf('data/allpaperinfo.hdf5')
paperinfo_df = paperinfo_df.loc[paperinfo_df['country_region'] != 'None'].copy(
)
paperinfo_df['afterChatGPT'] = (
    paperinfo_df['date'] >= gpt_release_date).astype(int)

df_bypaper_scored.merge

df_bypaper_scored = df_bypaper_scored.merge(
    paperinfo_df[['platform',
                  'title', 'date', 'category', 'country_region', 'authors',
                  'afterChatGPT']],
    right_on='_id',
    left_index=True
)
df_bypaper_scored_after = df_bypaper_scored.loc[
    df_bypaper_scored['afterChatGPT'] == 1
].copy()

In [ ]:
df_bypaper_scored_after.shape

In [ ]:
df_bypaper_scored_short_low_bino = df_bypaper_scored_after.loc[
    (df_bypaper_scored_after['bino_score_mean'] < df_bypaper_scored_after['bino_score_mean'].quantile(0.10)) & (
        df_bypaper_scored_after['n_tokens_k'] < 10
    )
].copy()
df_bypaper_scored_short_low_bino.shape

In [ ]:
declarations = []
for res in json.load(open('data/merged_declarations.json')):
    if res['has'] in ('N', 'Y'):
        declarations.append({
            '_id': res['_id'],
            'declaration': int(res['has'] == 'Y'),
        })
    else:
        declarations.append({
            '_id': res['_id'],
            'declaration': -1,
        })

declarations = pd.DataFrame(declarations, columns=[
                            '_id', 'declaration']).set_index('_id')
declarations.head()
df_bypaper_scored_after_declaration = df_bypaper_scored_after.merge(
    declarations,
    right_on='_id',
    left_index=True
)
print(df_bypaper_scored_after_declaration['declaration'].mean())
df_bypaper_scored_after_declaration.drop('text', axis=1).to_excel('data/paper_w_declarations.xlsx')
stats.pearsonr(
    df_bypaper_scored_after_declaration['declaration'],
    df_bypaper_scored_after_declaration['bino_score_min'],
)


In [ ]:
df_bypaper_scored_after_declaration.loc[
    df_bypaper_scored_after_declaration['declaration']==1.
].copy().shape[0]/len(df_bypaper_scored_after_declaration)